# Experiment 002: Aggressive Simulated Annealing

The pre-optimized submission is at a strong local optimum. Let's try more aggressive perturbations.

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import warnings
import random
import math
warnings.filterwarnings('ignore')

getcontext().prec = 30
scale_factor = 1

print('Libraries loaded successfully')

Libraries loaded successfully


In [2]:
# Tree polygon vertices (from the C++ code)
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_polygon(x, y, deg):
    """Get polygon vertices for a tree at position (x, y) with rotation deg."""
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    px = TX * c - TY * s + x
    py = TX * s + TY * c + y
    return Polygon(zip(px, py))

def check_overlap(poly1, poly2):
    """Check if two polygons overlap (not just touch)."""
    return poly1.intersects(poly2) and not poly1.touches(poly2)

def get_side(xs, ys, degs):
    """Calculate bounding box side length for a configuration."""
    all_px, all_py = [], []
    for x, y, deg in zip(xs, ys, degs):
        rad = np.radians(deg)
        c, s = np.cos(rad), np.sin(rad)
        px = TX * c - TY * s + x
        py = TX * s + TY * c + y
        all_px.extend(px)
        all_py.extend(py)
    return max(max(all_px) - min(all_px), max(all_py) - min(all_py))

def get_score(xs, ys, degs, n):
    """Calculate score for a configuration."""
    side = get_side(xs, ys, degs)
    return side * side / n

def has_any_overlap(xs, ys, degs):
    """Check if any trees overlap."""
    polys = [get_polygon(x, y, d) for x, y, d in zip(xs, ys, degs)]
    for i in range(len(polys)):
        for j in range(i + 1, len(polys)):
            if check_overlap(polys[i], polys[j]):
                return True
    return False

print('Helper functions defined')

Helper functions defined


In [3]:
# Load the pre-optimized submission
df = pd.read_csv('/home/code/preoptimized_submission.csv')

# Parse the submission
configs = {}
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    xs = np.array([float(str(x)[1:]) for x in group['x']])
    ys = np.array([float(str(y)[1:]) for y in group['y']])
    degs = np.array([float(str(d)[1:]) for d in group['deg']])
    configs[n] = {'xs': xs, 'ys': ys, 'degs': degs}

# Calculate initial total score
initial_score = sum(get_score(configs[n]['xs'], configs[n]['ys'], configs[n]['degs'], n) for n in range(1, 201))
print(f'Initial total score: {initial_score:.6f}')

Initial total score: 70.743774


In [4]:
# Per-N score breakdown
print('Per-N score breakdown (top 20 worst):')
scores = []
for n in range(1, 201):
    score = get_score(configs[n]['xs'], configs[n]['ys'], configs[n]['degs'], n)
    scores.append((n, score))

scores.sort(key=lambda x: -x[1])  # Sort by score descending
for n, score in scores[:20]:
    print(f'  N={n:3d}: {score:.6f}')

Per-N score breakdown (top 20 worst):
  N=  1: 0.661250
  N=  2: 0.450779
  N=  3: 0.434745
  N=  5: 0.416850
  N=  4: 0.416545
  N=  7: 0.399897
  N=  6: 0.399610
  N=  9: 0.387415
  N=  8: 0.385407
  N= 14: 0.380398
  N= 15: 0.379203
  N= 10: 0.376630
  N= 21: 0.376451
  N= 20: 0.376057
  N= 11: 0.375736
  N= 22: 0.375258
  N= 16: 0.374128
  N= 26: 0.373997
  N= 12: 0.372724
  N= 13: 0.372323


In [5]:
# Aggressive simulated annealing for a single N
def aggressive_sa(xs, ys, degs, n, max_iter=10000, T0=1.0, T_min=1e-6, alpha=0.995):
    """Aggressive simulated annealing with larger perturbations."""
    best_xs, best_ys, best_degs = xs.copy(), ys.copy(), degs.copy()
    best_score = get_score(xs, ys, degs, n)
    
    curr_xs, curr_ys, curr_degs = xs.copy(), ys.copy(), degs.copy()
    curr_score = best_score
    
    T = T0
    no_improve = 0
    
    for it in range(max_iter):
        # Choose a random tree
        i = random.randint(0, n - 1)
        
        # Save old values
        old_x, old_y, old_deg = curr_xs[i], curr_ys[i], curr_degs[i]
        
        # Choose move type
        move_type = random.choice(['translate', 'rotate', 'both'])
        
        if move_type == 'translate' or move_type == 'both':
            # Larger translation perturbation
            step = random.choice([0.01, 0.005, 0.002, 0.001, 0.0005])
            dx = random.gauss(0, step)
            dy = random.gauss(0, step)
            curr_xs[i] += dx
            curr_ys[i] += dy
        
        if move_type == 'rotate' or move_type == 'both':
            # Larger rotation perturbation
            angle_step = random.choice([10, 5, 2, 1, 0.5])
            curr_degs[i] += random.gauss(0, angle_step)
            curr_degs[i] = curr_degs[i] % 360
        
        # Check for overlaps
        if has_any_overlap(curr_xs, curr_ys, curr_degs):
            # Reject move
            curr_xs[i], curr_ys[i], curr_degs[i] = old_x, old_y, old_deg
            continue
        
        # Calculate new score
        new_score = get_score(curr_xs, curr_ys, curr_degs, n)
        
        # Accept or reject
        delta = new_score - curr_score
        if delta < 0 or random.random() < math.exp(-delta / T):
            curr_score = new_score
            if new_score < best_score:
                best_xs, best_ys, best_degs = curr_xs.copy(), curr_ys.copy(), curr_degs.copy()
                best_score = new_score
                no_improve = 0
        else:
            # Reject move
            curr_xs[i], curr_ys[i], curr_degs[i] = old_x, old_y, old_deg
            no_improve += 1
        
        # Cool down
        T = max(T * alpha, T_min)
        
        # Early stopping if no improvement for a while
        if no_improve > 1000:
            break
    
    return best_xs, best_ys, best_degs, best_score

print('Aggressive SA function defined')

Aggressive SA function defined


In [6]:
# Try aggressive SA on the worst N values
improved_configs = {}
total_improvement = 0

print('Running aggressive SA on worst N values...')
for n, orig_score in scores[:30]:  # Top 30 worst
    xs = configs[n]['xs'].copy()
    ys = configs[n]['ys'].copy()
    degs = configs[n]['degs'].copy()
    
    new_xs, new_ys, new_degs, new_score = aggressive_sa(xs, ys, degs, n, max_iter=5000)
    
    if new_score < orig_score - 1e-9:
        improvement = orig_score - new_score
        total_improvement += improvement
        improved_configs[n] = {'xs': new_xs, 'ys': new_ys, 'degs': new_degs}
        print(f'  N={n:3d}: {orig_score:.6f} -> {new_score:.6f} (improved by {improvement:.6f})')
    else:
        improved_configs[n] = configs[n]

print(f'\nTotal improvement: {total_improvement:.6f}')

Running aggressive SA on worst N values...



Total improvement: 0.000000


In [7]:
# Calculate final score
final_configs = configs.copy()
for n, cfg in improved_configs.items():
    final_configs[n] = cfg

final_score = sum(get_score(final_configs[n]['xs'], final_configs[n]['ys'], final_configs[n]['degs'], n) for n in range(1, 201))
print(f'Initial score: {initial_score:.6f}')
print(f'Final score: {final_score:.6f}')
print(f'Improvement: {initial_score - final_score:.6f}')

Initial score: 70.743774
Final score: 70.743774
Improvement: 0.000000


In [8]:
# Generate submission if improved
if final_score < initial_score - 1e-9:
    rows = []
    for n in range(1, 201):
        cfg = final_configs[n]
        for i in range(n):
            rows.append({
                'id': f'{n:03d}_{i}',
                'x': f's{cfg["xs"][i]}',
                'y': f's{cfg["ys"][i]}',
                'deg': f's{cfg["degs"][i]}'
            })
    
    df_out = pd.DataFrame(rows)
    df_out.to_csv('/home/submission/submission.csv', index=False)
    print(f'Saved improved submission with score {final_score:.6f}')
else:
    # Keep the original pre-optimized submission
    import shutil
    shutil.copy('/home/code/preoptimized_submission.csv', '/home/submission/submission.csv')
    print(f'No improvement found, keeping original submission with score {initial_score:.6f}')

No improvement found, keeping original submission with score 70.743774
